In [1]:
import pandas as pd
import numpy as np

In [2]:
#from matplotlib import pyplot as plt

# dataframe preparation

In [3]:
data=pd.read_csv('data/Dataset/Dataset/Training/Features_Variant_1.csv', header=None).drop_duplicates()

In [4]:
print(data.iloc[:,37].unique()) #post promotion has only zero values we can drop this col because it isn't informative

[0]


In [5]:
#get and fit features names
features_names=np.array(['page_likes_num',#1
               'page_checkins',#2
               'page_talking_about',#3
                'page_cat',#4
                'page_statistics',#5-29 #mean, avg etc.
                'comments_num_before_base_time',#30
                'comments_num_in_last_24_hours',#31 #last day
                'comments_num_in_last_48_to_24_hours',#32 #day before last
                'comments_num_in_first_24_hours',#33
                'comments_difference_in_last_two_days', #34 (32-31)
                'base_time', #35
                'character_num_in_post', #36
                'share_num',#37
                'post_promotion', #38 binary
                'h_local', #39 This describes the H hrs, for which we have the target variable/ comments received. 
                'post_published_weekday', #40-46 This represents the day(Sunday...Saturday) on which the post was published. 
                'base_ditetime_weekday', #47-53 This represents the day(Sunday...Saturday) on selected base Date/Time. 
                'target' #54 The no of comments in next H hrs(H is given in Feature no 39).                
               ])

for index in range(5,29):
    features_names=np.insert(features_names, index, features_names[4]+'_'+str(index-4))
    
weekday=('sunday', 'monday','tuesday', 'wednesday', 'thursday', 'friday', 'saturday')    

for index in range(40,47):
    features_names=np.insert(features_names,index, features_names[39]+'_'+ weekday[index-40])
features_names=np.delete(features_names, 39)

for index in range(47,54):
    features_names=np.insert(features_names,index, features_names[46]+'_'+ weekday[index-47])
features_names=np.delete(features_names, 46)

data.columns=features_names

In [6]:
y=data.nunique(axis=0).sort_values()
print(y.head())

post_promotion                      1
post_published_weekday_sunday       2
post_published_weekday_monday       2
post_published_weekday_tuesday      2
post_published_weekday_wednesday    2
dtype: int64


In [7]:
data=data.drop(columns=['post_promotion'])

# data preparation

In [8]:
#'page_cat' to dummy because it is a category
data=pd.concat([data,pd.get_dummies(data['page_cat'], prefix='page_cat')], axis=1)
data=data.drop(columns=['page_cat'])

In [9]:
def normalize(df):
    std=np.std(df, ddof=1).replace(0, 1)
    return (df-np.mean(df))/std

# learning

In [10]:
def custom_RMSE(target, pred, gamma, w):
    return np.sqrt(
        np.sum((target-pred)**2)/len(pred)+gamma*np.linalg.norm(w)
    )

def get_para_w(old_w, df, pred, lam, gamma, terms_num, iter_num):#, start_ind):
    indices= np.random.choice(df.shape[1]-1, terms_num, replace=False)
    #indices=np.array(range(start_ind, start_ind+terms_num))%df.shape[0]
    #start_ind=start_ind+terms_num
    old_w=old_w+2*lam*(
          (np.dot((df.loc[indices,'target']-pred[indices]).values,df.iloc[indices,:-1].values))/(len(pred))#*iter_num)
        )
    return old_w, indices#, start_ind

def get_para_w0(old_w0, df, pred, lam, gamma, indices, iter_num):
    return old_w0+2*lam*(
        (np.sum(df.loc[indices,'target']-pred[indices]))/(len(pred))#*iter_num)
        )
def get_prediction(w,w0,x_df):
    return np.dot(x_df, w)+w0

def RMSE_plot(RMSE_arr):
    plt.figure(figsize=(15,7))   
    plt.plot(RMSE_arr)
    plt.xlabel('iteration number')
    plt.ylabel('RMSE value')
    plt.title('algo scores')
    plt.show()
    
def gradient_descend(df, lam, gamma, terms_num, max_iter):
    df=df.reset_index(drop=True)
    w=np.array([0]*(df.shape[1]-1), dtype=float) #one column is the target
    w0=0.0
    prediction=get_prediction(w,w0,df.iloc[:,:-1])
    minRMSE=custom_RMSE(df['target'], prediction, gamma, w)
    #RMSE_arr=np.array([minRMSE], dtype=float)
    params=np.append(w,w0)
    best_params=params
    best_pred=prediction

    curr_err=10
    err=10e-5
    
    iter_num=0
    start_ind=0
    while (curr_err>err) & (iter_num<max_iter) :    
        iter_num+=1
        w, indices=get_para_w(w, df, prediction, lam, gamma, terms_num, iter_num)
        w0=get_para_w0(w0, df, prediction, lam, gamma, indices, iter_num)
        
        #first variant of stopping criterion:
        curr_err=prediction.copy()
        prediction=get_prediction(w,w0,df.iloc[:,:-1])
        curr_err=np.linalg.norm(curr_err-prediction)#||y(k+1)-y(k)||
        
        """
        #another stopping criterion:
        curr_err=params.copy() #old params        
        params=np.append(w,w0)
        curr_err=np.linalg.norm(curr_err-params)#||w(k+1)-w(k)||
        """
        params=np.append(w,w0)
        #RMSE_arr=np.append(RMSE_arr,custom_RMSE(df['target'], prediction, gamma, w))
        if custom_RMSE(df['target'], prediction, gamma, w)<minRMSE:
            minRMSE=custom_RMSE(df['target'], prediction, gamma, w)
            best_params=params.copy()
            best_pred=prediction
            
    print('Iterations number is:' +str(iter_num))
    if iter_num==max_iter:
        print('The maximum number of iterations was reached.')
    print('RMSE is:'+str(minRMSE))
    print('R2 is:'+str(custom_R2(df.iloc[:,-1], best_pred)))
    
    #RMSE_plot(RMSE_arr)        
    return best_params

In [11]:
#set algo params:
lam=1e-2
gamma=2.0
terms_num=10
max_iter=1e4

# Cross-val

In [12]:
def custom_R2(target, pred):
    return 1-(np.sum((target-pred)**2))/(np.sum((target-np.mean(target))**2))

In [13]:
res=pd.DataFrame()

parts=range(1,6)
idxs=[round(data.shape[0]/5)*(i-1) for i in range(1,6)]
idxs.append(data.shape[0])

for part in parts:
    #separate dataset
    test_part=data[idxs[part-1]:idxs[part]]
    train_part=data.drop(data.index[idxs[part-1]:idxs[part]])
    #features normalization (xi-mean)/std_err:
    train_part.iloc[:,:-1]=normalize(train_part.iloc[:,:-1])
    params=gradient_descend(train_part, lam, gamma, terms_num, max_iter) #model    
    test_pred=get_prediction(params[:-1],params[-1],test_part.iloc[:,:-1])
    RMSE=custom_RMSE(test_part['target'], test_pred, gamma, params[:-1])
    r2=custom_R2(test_part['target'], test_pred)
    params=np.insert(params, 0 , [RMSE,r2])
    res[part]=params

Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9907683896833979
R2 is:-4.47272026633581
Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9919830697327908
R2 is:-9.380949049129503
Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.99283866471241
R2 is:-9.812493711650392
Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9914484949059928
R2 is:-10.454794209568103
Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9941627535647847
R2 is:-inf


C:\Users\risam\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


In [14]:
idxs=np.array(['w'+str(i) for i in range(1,res.shape[0]-2)])
idxs=np.append(idxs,'w0')
idxs=np.insert(idxs, 0, ['RMSE', 'r2'])
res.index=idxs

res['mean']=np.mean(res, axis=1)
res['std']=np.std(res.iloc[:,:-1], ddof=1, axis=1)

In [15]:
res

,1,2,3,4,5,mean,std
RMSE,2700.227102,707.663618,450.699493,5026.769559,623.067663,1901.685487,1972.924941
r2,-4632.325388,-410.970603,-137.791291,-19817.617495,-506.445994,-5101.030154,8434.130502
w1,-0.000500,0.000198,0.000183,0.000368,0.000140,0.000078,0.000334
w2,-0.000755,0.000482,0.000416,0.000489,0.000353,0.000197,0.000535
w3,-0.000407,0.000805,0.000724,0.000912,0.000627,0.000532,0.000535
w4,-0.000256,0.000171,0.000138,0.000156,0.000115,0.000065,0.000180
w5,0.002019,-0.001428,-0.001271,-0.001455,-0.001111,-0.000649,0.001498
w6,0.000976,0.001054,0.000972,0.001033,0.000776,0.000962,0.000110
w7,0.000216,0.001008,0.000959,0.001003,0.000738,0.000785,0.000337
w8,0.001720,-0.000012,-0.000035,-0.000098,-0.000034,0.000308,0.000790


In [16]:
#this part is for comparison
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
model=LinearRegression()
scores = cross_val_score(model, data.iloc[:,:-1], data.iloc[:,-1], cv=5, scoring='r2')
print('R2 scores for each fold is:')
print(scores)

R2 scores for each fold is:
[ 0.          0.          0.          0.         -0.00725797]
